### CUNY Data 620 - Web Analytics, Summer 2020  
**Final Project: Twitter Pull**   
**Prof:** Alain Ledon  
**Members:** Misha Kollontai, Amber Ferger, Zach Alexander, Subhalaxmi Rout 

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
import datetime
import time
import math
import GetOldTweets3 as got
warnings.filterwarnings('ignore')

### Functions

We'll define the following functions:
* **perdelta**: Based on a [stackoverflow](https://stackoverflow.com/questions/10688006/generate-a-list-of-datetimes-between-an-interval) thread, this will be used to generate a list of date ranges for our twitter pull. 
* **getTweets**: This will be used to pull the tweets. 

In [2]:
################ date function
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta
        
################ get tweets function 
def getTweets(city, startDate, endDate):
    n = 100
    
    tweetCriteria = got.manager.TweetCriteria().setQuerySearch('COVID')\
    .setSince(startDate)\
    .setUntil(endDate)\
    .setMaxTweets(n)\
    .setNear(city)
    
    ls = []
    
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    print(len(tweets))
    for i in tweets:
        ls.append([i.text, i.hashtags, i.date, i.username, city, startDate, endDate])
    
    return(ls)

### Twitter Data

#### Largest City by State

* Read in a list of the top 1000 [cities]([https://public.opendatasoft.com/explore/dataset/1000-largest-us-cities-by-population-with-geographic-coordinates/table/?sort=-rank]) in the US
* Select top city by state, extract geocoordinates
* Split the geocoordinates data into 2 lists so that we can run on 2 separate machines (there is a max of 15 requests per 15 minutes)

In [3]:
# read in cities doc, select top city from each 
# https://stackoverflow.com/questions/50415632/how-to-select-top-n-row-from-each-group-after-group-by-in-pandas
allData = pd.read_csv('largeCities.csv', delimiter=';')
final_cities = allData.sort_values(by = ['State', 'Population'], ascending=False).groupby(['State'], sort=False).head(1)
coords = final_cities['Coordinates'].values.tolist()

In [4]:
# splitting the coordinates into 2 lists
mid = math.floor(len(coords)/2)
coords1 = coords[0:mid]
coords2 = coords[mid:]

#### Date Ranges
Next, we'll generate date ranges for pull. Each range will represent 2 weeks, defined as Sunday - Saturday. The total span of the analysis will go from **3/8/2020** to **7/11/2020**.

In [7]:
all_dates = []
for result in perdelta(datetime.date(2020, 3, 8), datetime.date(2020, 7, 6), datetime.timedelta(days=7)):  
    nextWk = result + datetime.timedelta(days=6)
    startDt = result.strftime("%Y-%m-%d")
    endDt = nextWk.strftime("%Y-%m-%d")   
    all_dates.append((startDt,endDt))
    
#all_dates = [(datetime.date(2020, 3, 8).strftime("%Y-%m-%d"), datetime.date(2020, 7, 15).strftime("%Y-%m-%d"))]
#all_dates

#### Pull Tweets

In [8]:
# Cycle through all cities
finalList = []

for i,c in enumerate(coords1):
    print(i)
    if (i+1) % 2 == 0:
        time.sleep(600) # wait 10 min before continuing
    for d in all_dates:
        ls = getTweets(c,d[0],d[1])
        [finalList.append(x) for x in ls]

0
3
4
3
5
6
3
2
3
3
1
0
2
2
2
1
5
6
30
1
85
100
100
100
100
100
86
88
76
87
77
73
57
62
59
64
92
100
2
20
25
31
32
20
16
21
16
16
13
12
7
5
6
11
10
29
100
3
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
4
41
100
100
100
99
78
61
58
65
76
66
82
47
51
45
60
62
100
5
19
26
21
35


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



An error occured during an HTTP request: [WinError 10054] An existing connection was forcibly closed by the remote host
Try to open in browser: https://twitter.com/search?q=COVID%20near%3A%2244.4758825%2C-73.212072%22%20within%3A15mi%20since%3A2020-04-05%20until%3A2020-04-11&src=typd
Traceback (most recent call last):
  File "c:\users\aferg\desktop\lib\site-packages\GetOldTweets3\manager\TweetManager.py", line 344, in getJsonResponse
    jsonResponse = response.read()
  File "c:\users\aferg\desktop\lib\http\client.py", line 467, in read
    s = self._safe_read(self.length)
  File "c:\users\aferg\desktop\lib\http\client.py", line 608, in _safe_read
    data = self.fp.read(amt)
  File "c:\users\aferg\desktop\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
  File "c:\users\aferg\desktop\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "c:\users\aferg\desktop\lib\ssl.py", line 1099, in read
    return self._sslobj.read(len, buffer)
C

SystemExit: 

In [9]:
df = pd.DataFrame(finalList, columns = ['TEXT', 'HASHTAGS', 'TWEET_DATE', 'USERNAME', 'COORDS', 'WEEK_START', 'WEEK_END'])

In [10]:
finalCsv = pd.merge(left=df, right=final_cities, left_on='COORDS', right_on='Coordinates')
finalCsv = finalCsv.drop(columns=['Rank', 'Growth From 2000 to 2013', 'Coordinates'])
finalCsv.to_csv('csvFinal.csv')